In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path
import gc

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from thc_net.classifier import ThcNetClassifier
from thc_net.input_utils import prepare_input_data

# Download census-income dataset

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
dataset_name = 'census-income'
out = Path(os.getcwd().rsplit("/",  1)[0]+'/data/'+dataset_name+'.csv')

In [ ]:
out.parent.mkdir(parents=True, exist_ok=True)
if out.exists():
    print("File already exists.")
else:
    print("Downloading file...")
    wget.download(url, out.as_posix())

# Load data and split

In [ ]:
train = pd.read_csv(out)
target = ' <=50K'
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [ ]:
Y = train[target].values
X = train.drop(columns=['Set'] + [target])

In [ ]:
ratio = 0.005

In [ ]:
n_unique = X.nunique()
ratios = (n_unique / X.shape[0]) < 0.005
cat_idxs = np.argwhere(X.columns.isin(X.columns[ratios | (X.dtypes == 'object')])).ravel()
cat_dims = n_unique[cat_idxs].values + X.isnull().sum()[cat_idxs].values + 1
del n_unique, ratios, train

In [ ]:
gc.collect()

# Training

In [ ]:
tg_enc = LabelEncoder()
Y = tg_enc.fit_transform(Y)

In [ ]:
X_train = X.values[train_indices]
y_train = Y[train_indices]

X_valid = X.values[valid_indices]
y_valid = Y[valid_indices]

X_test = X.values[test_indices]
y_test = Y[test_indices]

In [ ]:
X_train.shape

In [ ]:
X_train_prep, encoders = prepare_input_data(X_train, cat_idxs)
X_valid_prep, _ = prepare_input_data(X_valid, cat_idxs, encoders=encoders, fit=False)
X_test_prep, _ = prepare_input_data(X_test, cat_idxs, encoders=encoders, fit=False)

# Network parameters

In [ ]:
%%time

model = ThcNetClassifier(
    n_layer=3,
    mul_input=8, 
    metrics=['AUC'],
    cat_idxs=cat_idxs,
    cat_emb_dims=cat_dims,
    dropout=0.05,
    normalize=False,
    max_emb=20,
    patience=30
)

history = model.fit(
        X=X_train_prep, 
        y=y_train,
        X_valid=X_valid_prep,
        y_valid=y_valid,
        batch_size=1024,
        epochs=10000,
        verbose=2,
)

In [ ]:
%%time

model_snn = ThcNetClassifier(
    n_layer=3,
    mul_input=8, 
    metrics=['AUC'],
    cat_idxs=cat_idxs,
    cat_emb_dims=cat_dims,
    dropout=0.05,
    normalize=False,
    max_emb=20,
    patience=30,
    use_snn=True,
    noise=None
)

history_snn = model_snn.fit(
        X=X_train_prep, 
        y=y_train,
        X_valid=X_valid_prep,
        y_valid=y_valid,
        batch_size=1024,
        epochs=10000,
        verbose=2,
)

In [ ]:
model.network.summary()

In [ ]:
model_snn.network.summary()

### Predictions

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['AUC'])
plt.plot(history.history['val_AUC'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

#history = model.fit(x, y, validation_split=0.25, epochs=50, batch_size=16, verbose=1)

# Plot training & validation accuracy values
plt.plot(history_snn.history['AUC'])
plt.plot(history_snn.history['val_AUC'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history_snn.history['loss'])
plt.plot(history_snn.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict_proba(X_test_prep)
test_auc = roc_auc_score(y_score=y_pred[:,1], y_true=y_test)
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

In [ ]:
y_pred = model_snn.predict_proba(X_test_prep)
test_auc = roc_auc_score(y_score=y_pred[:,1], y_true=y_test)
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")